In [1]:
import pandas as pd

### Данные пользователей 

user_info = pd.read_sql(
    """SELECT * FROM public.user_data""",

    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)
user_info.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [2]:
posts_info = pd.read_sql(
    """SELECT * FROM public.post_text_df""",

    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)
posts_info.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [3]:
# 77 миллионов записей
count_feed_data = pd.read_sql(
    """SELECT count(*) FROM public.feed_data""",

    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

count_feed_data.head()

,count
0,76892800


In [4]:
feed_data = pd.read_sql(
    """SELECT * FROM public.feed_data LIMIT 5000000""",
    
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-10-06 22:28:19,75106,2061,view,0
1,2021-10-06 22:29:29,75106,3598,view,0
2,2021-10-06 22:30:33,75106,685,view,0
3,2021-10-06 22:31:01,75106,5921,view,0
4,2021-10-06 22:32:42,75106,3110,view,0


In [5]:
feed_data[feed_data.action!='view']

,timestamp,user_id,post_id,action,target
10,2021-10-07 11:11:09,75106,4438,like,0
21,2021-10-07 11:25:26,75106,2888,like,0
23,2021-10-07 11:28:00,75106,4214,like,0
28,2021-10-07 11:32:57,75106,4834,like,0
50,2021-10-08 09:30:28,75106,4201,like,0
...,...,...,...,...,...
4999937,2021-11-29 17:56:31,68631,718,like,0
4999941,2021-11-29 18:01:29,68631,1780,like,0
4999948,2021-11-29 18:10:10,68631,1720,like,0
4999973,2021-12-04 15:25:21,68631,535,like,0


In [6]:
feed_data = feed_data[feed_data.action=='view']

feed_data.head()


,timestamp,user_id,post_id,action,target
0,2021-10-06 22:28:19,75106,2061,view,0
1,2021-10-06 22:29:29,75106,3598,view,0
2,2021-10-06 22:30:33,75106,685,view,0
3,2021-10-06 22:31:01,75106,5921,view,0
4,2021-10-06 22:32:42,75106,3110,view,0


In [7]:
### Фиксированный наблор признаков по user_id
user_info

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [8]:
### Придумаем эмбеддинг для post_id

posts_info

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [9]:
import re
import string
import unicodedata
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

wnl = WordNetLemmatizer()

### Препроцессинг

def preprocessing(line, token=wnl):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    line = line.replace("\n\n", " ").replace("\n", " ")
    line = ' '.join([token.lemmatize(x) for x in line.split(' ')])
    return line

tfidf = TfidfVectorizer(
    stop_words='english',
    preprocessor=preprocessing
)

In [10]:
tfidf_data = (
    tfidf.fit_transform(posts_info['text']).toarray()
)

tfidf_data

C:\Users\nikul\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'wa'] not in stop_words.
  warnings.warn(


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.13273932, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05061394, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
tfidf_data = pd.DataFrame(
    tfidf_data,
    index = posts_info.post_id,
    columns=tfidf.get_feature_names_out()
)

tfidf_data

,00,000,0001,000bn,000m,000s,000th,001,001and,001st,...,𝓫𝓮,𝓫𝓮𝓽𝓽𝓮𝓻,𝓬𝓸𝓾𝓻𝓽𝓼,𝓱𝓮𝓪𝓻𝓲𝓷𝓰,𝓶𝓪𝔂,𝓹𝓱𝔂𝓼𝓲𝓬𝓪𝓵,𝓼𝓸𝓸𝓷𝓮𝓻,𝓼𝓾𝓫𝓸𝓻𝓭𝓲𝓷𝓪𝓽𝓮,𝓽𝓱𝓮,𝓽𝓸
post_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.132739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.050614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7316,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7317,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### Генерация фичи на основне Tfidf

posts_info['TotalTfidf'] = tfidf_data.sum(axis=1).reset_index()[0]
posts_info['MaxTfidf'] = tfidf_data.max(axis=1).reset_index()[0]
posts_info['MeanTfidf'] = tfidf_data.mean(axis=1).reset_index()[0]

posts_info.head()

,post_id,text,topic,TotalTfidf,MaxTfidf,MeanTfidf
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8.748129,0.495805,0.000190
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11.878472,0.308003,0.000258
2,3,Asian quake hits European shares\n\nShares in ...,business,12.675530,0.261799,0.000276
3,4,India power shares jump on debut\n\nShares in ...,business,6.622786,0.537713,0.000144
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,6.352096,0.420251,0.000138


In [13]:
[f"DistanceTo{ith}thCluster" for ith in range(1, 16)]

['DistanceTo1thCluster',
 'DistanceTo2thCluster',
 'DistanceTo3thCluster',
 'DistanceTo4thCluster',
 'DistanceTo5thCluster',
 'DistanceTo6thCluster',
 'DistanceTo7thCluster',
 'DistanceTo8thCluster',
 'DistanceTo9thCluster',
 'DistanceTo10thCluster',
 'DistanceTo11thCluster',
 'DistanceTo12thCluster',
 'DistanceTo13thCluster',
 'DistanceTo14thCluster',
 'DistanceTo15thCluster']

In [14]:
### Пытаемся кластеризировать тексты 

from sklearn.decomposition import PCA

centered = tfidf_data - tfidf_data.mean()

pca = PCA(n_components=20)
pca_decomp = pca.fit_transform(centered)

from sklearn.cluster import KMeans 

kmeans = KMeans(n_clusters=15, random_state=0).fit(pca_decomp)

posts_info['TextCluster'] = kmeans.labels_
dists_columns = ['DistanceTo1thCluster',
                 'DistanceTo2thCluster',
                 'DistanceTo3thCluster',
                 'DistanceTo4thCluster',
                 'DistanceTo5thCluster',
                 'DistanceTo6thCluster',
                 'DistanceTo7thCluster',
                 'DistanceTo8thCluster',
                 'DistanceTo9thCluster',
                 'DistanceTo10thCluster',
                 'DistanceTo11thCluster',
                 'DistanceTo12thCluster',
                 'DistanceTo13thCluster',
                 'DistanceTo14thCluster',
                 'DistanceTo15thCluster']

dists_df = pd.DataFrame(
    data = kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,0.498763,0.438866,0.478928,0.533634,0.274267,0.501817,0.549847,0.407473,0.441331,0.465443,0.444900,0.529236,0.505747,0.522130,0.437839
1,0.371631,0.291801,0.341899,0.277830,0.209640,0.375288,0.450040,0.173449,0.292427,0.329292,0.282600,0.414625,0.426678,0.413365,0.259837
2,0.397930,0.313296,0.362400,0.480566,0.097027,0.389489,0.484424,0.279320,0.315376,0.356818,0.326329,0.431981,0.427459,0.435588,0.309584
3,0.323621,0.256363,0.321060,0.432745,0.103342,0.352320,0.447335,0.200471,0.263464,0.299496,0.287838,0.385666,0.403035,0.399316,0.257896
4,0.294497,0.160849,0.260119,0.394337,0.174729,0.296027,0.405016,0.139676,0.179389,0.234276,0.219992,0.343986,0.352692,0.391472,0.182880


In [15]:
posts_info = pd.concat((posts_info, dists_df), axis=1)

posts_info.head()

,post_id,text,topic,TotalTfidf,MaxTfidf,MeanTfidf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,...,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8.748129,0.495805,0.000190,4,0.498763,0.438866,0.478928,...,0.501817,0.549847,0.407473,0.441331,0.465443,0.444900,0.529236,0.505747,0.522130,0.437839
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11.878472,0.308003,0.000258,7,0.371631,0.291801,0.341899,...,0.375288,0.450040,0.173449,0.292427,0.329292,0.282600,0.414625,0.426678,0.413365,0.259837
2,3,Asian quake hits European shares\n\nShares in ...,business,12.675530,0.261799,0.000276,4,0.397930,0.313296,0.362400,...,0.389489,0.484424,0.279320,0.315376,0.356818,0.326329,0.431981,0.427459,0.435588,0.309584
3,4,India power shares jump on debut\n\nShares in ...,business,6.622786,0.537713,0.000144,4,0.323621,0.256363,0.321060,...,0.352320,0.447335,0.200471,0.263464,0.299496,0.287838,0.385666,0.403035,0.399316,0.257896
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,6.352096,0.420251,0.000138,7,0.294497,0.160849,0.260119,...,0.296027,0.405016,0.139676,0.179389,0.234276,0.219992,0.343986,0.352692,0.391472,0.182880


In [16]:
df = pd.merge(feed_data,
              posts_info,
              on='post_id',
              how='left')

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TotalTfidf,MaxTfidf,MeanTfidf,...,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,2021-10-06 22:28:19,75106,2061,view,0,Spam e-mails tempt net shoppers\n\nComputer us...,tech,9.209704,0.562307,0.000200,...,0.501399,0.575248,0.357967,0.446605,0.471221,0.468062,0.535731,0.539659,0.514619,0.463544
1,2021-10-06 22:29:29,75106,3598,view,0,Together we make a difference 💚🕊️\nWith the ge...,covid,3.175461,0.432053,0.000069,...,0.289203,0.416136,0.206734,0.164932,0.140198,0.218359,0.283761,0.340108,0.420281,0.243512
2,2021-10-06 22:30:33,75106,685,view,0,US afflicted with awards fatigue\n\nThe film w...,entertainment,12.946364,0.276067,0.000281,...,0.502587,0.189052,0.436137,0.446949,0.509137,0.470171,0.568516,0.551820,0.558759,0.439667
3,2021-10-06 22:31:01,75106,5921,view,0,Following the success of the (awful) Gilligans...,movie,9.395559,0.397973,0.000204,...,0.110518,0.392267,0.281720,0.158900,0.321637,0.294147,0.409135,0.399375,0.442200,0.275286
4,2021-10-06 22:32:42,75106,3110,view,0,.portland riot park had no covid rules. Rules ...,covid,3.111524,0.389931,0.000068,...,0.336343,0.443126,0.258143,0.239462,0.219780,0.291137,0.143504,0.400725,0.439255,0.302063


In [17]:
### Датафрейм со всеми новыми фичами

df = pd.merge(df,
              user_info,
              on='user_id',
              how='left')

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TotalTfidf,MaxTfidf,MeanTfidf,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,gender,age,country,city,exp_group,os,source
0,2021-10-06 22:28:19,75106,2061,view,0,Spam e-mails tempt net shoppers\n\nComputer us...,tech,9.209704,0.562307,0.000200,...,0.539659,0.514619,0.463544,1,16,Russia,Voronezh,4,Android,ads
1,2021-10-06 22:29:29,75106,3598,view,0,Together we make a difference 💚🕊️\nWith the ge...,covid,3.175461,0.432053,0.000069,...,0.340108,0.420281,0.243512,1,16,Russia,Voronezh,4,Android,ads
2,2021-10-06 22:30:33,75106,685,view,0,US afflicted with awards fatigue\n\nThe film w...,entertainment,12.946364,0.276067,0.000281,...,0.551820,0.558759,0.439667,1,16,Russia,Voronezh,4,Android,ads
3,2021-10-06 22:31:01,75106,5921,view,0,Following the success of the (awful) Gilligans...,movie,9.395559,0.397973,0.000204,...,0.399375,0.442200,0.275286,1,16,Russia,Voronezh,4,Android,ads
4,2021-10-06 22:32:42,75106,3110,view,0,.portland riot park had no covid rules. Rules ...,covid,3.111524,0.389931,0.000068,...,0.400725,0.439255,0.302063,1,16,Russia,Voronezh,4,Android,ads


In [18]:
df['hour'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.hour)
df['month'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.month)

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TotalTfidf,MaxTfidf,MeanTfidf,...,DistanceTo15thCluster,gender,age,country,city,exp_group,os,source,hour,month
0,2021-10-06 22:28:19,75106,2061,view,0,Spam e-mails tempt net shoppers\n\nComputer us...,tech,9.209704,0.562307,0.000200,...,0.463544,1,16,Russia,Voronezh,4,Android,ads,22,10
1,2021-10-06 22:29:29,75106,3598,view,0,Together we make a difference 💚🕊️\nWith the ge...,covid,3.175461,0.432053,0.000069,...,0.243512,1,16,Russia,Voronezh,4,Android,ads,22,10
2,2021-10-06 22:30:33,75106,685,view,0,US afflicted with awards fatigue\n\nThe film w...,entertainment,12.946364,0.276067,0.000281,...,0.439667,1,16,Russia,Voronezh,4,Android,ads,22,10
3,2021-10-06 22:31:01,75106,5921,view,0,Following the success of the (awful) Gilligans...,movie,9.395559,0.397973,0.000204,...,0.275286,1,16,Russia,Voronezh,4,Android,ads,22,10
4,2021-10-06 22:32:42,75106,3110,view,0,.portland riot park had no covid rules. Rules ...,covid,3.111524,0.389931,0.000068,...,0.302063,1,16,Russia,Voronezh,4,Android,ads,22,10


In [19]:
df = df.drop([
    # 'timestamp',
    'action',
    'text',
],
             axis = 1)

df = df.set_index(['user_id', 'post_id'])

df.head()

timestamp  target          topic  TotalTfidf  \
user_id post_id                                                          
75106   2061    2021-10-06 22:28:19       0           tech    9.209704   
        3598    2021-10-06 22:29:29       0          covid    3.175461   
        685     2021-10-06 22:30:33       0  entertainment   12.946364   
        5921    2021-10-06 22:31:01       0          movie    9.395559   
        3110    2021-10-06 22:32:42       0          covid    3.111524   

                 MaxTfidf  MeanTfidf  TextCluster  DistanceTo1thCluster  \
user_id post_id                                                           
75106   2061     0.562307   0.000200            7              0.507963   
        3598     0.432053   0.000069            1              0.252389   
        685      0.276067   0.000281            6              0.552661   
        5921     0.397973   0.000204            5              0.358579   
        3110     0.389931   0.000068           11              0.302570   

                 DistanceTo2thCluster  DistanceTo3thCluster  ...  \
user_id post_id                                              ...   
75106   2061                 0.444204              0.481784  ...   
        3598                 0.032238              0.256382  ...   
        685                  0.490475              0.411906  ...   
        5921                 0.266711              0.239152  ...   
        3110                 0.159409              0.309820  ...   

                 DistanceTo15thCluster  gender  age  country      city  \
user_id post_id                                                          
75106   2061                  0.463544       1   16   Russia  Voronezh   
        3598                  0.243512       1   16   Russia  Voronezh   
        685                   0.439667       1   16   Russia  Voronezh   
        5921                  0.275286       1   16   Russia  Voronezh   
        3110                  0.302063       1   16   Russia  Voronezh   

                 exp_group       os  source  hour  month  
user_id post_id                                           
75106   2061             4  Android     ads    22     10  
        3598             4  Android     ads    22     10  
        685              4  Android     ads    22     10  
        5921             4  Android     ads    22     10  
        3110             4  Android     ads    22     10  

[5 rows x 31 columns]

In [20]:
max(df.timestamp), min(df.timestamp)

(Timestamp('2021-12-29 23:39:35'), Timestamp('2021-10-01 06:06:44'))

In [21]:
# За отсечку возьмем 2021-12-15

df_train = df[df.timestamp < '2021-12-15']
df_test = df[df.timestamp >= '2021-12-15']

df_train = df_train.drop('timestamp', axis = 1)
df_test = df_test.drop('timestamp', axis=1)

X_train = df_train.drop('target', axis=1)
X_test = df_test.drop('target', axis=1)

y_train = df_train['target']
y_test = df_test['target']

y_train.shape, y_test.shape

((3732580,), (737625,))

In [22]:
X_train

topic  TotalTfidf  MaxTfidf  MeanTfidf  TextCluster  \
user_id post_id                                                                
75106   2061              tech    9.209704  0.562307   0.000200            7   
        3598             covid    3.175461  0.432053   0.000069            1   
        685      entertainment   12.946364  0.276067   0.000281            6   
        5921             movie    9.395559  0.397973   0.000204            5   
        3110             covid    3.111524  0.389931   0.000068           11   
...                        ...         ...       ...        ...          ...   
68631   5632             movie    7.227848  0.280075   0.000157            1   
        3359             covid    3.509983  0.418316   0.000076            0   
        3361             covid    3.265125  0.491853   0.000071            9   
        3314             covid    3.634019  0.398439   0.000079           11   
        5124             movie    6.594952  0.239079   0.000143            8   

                 DistanceTo1thCluster  DistanceTo2thCluster  \
user_id post_id                                               
75106   2061                 0.507963              0.444204   
        3598                 0.252389              0.032238   
        685                  0.552661              0.490475   
        5921                 0.358579              0.266711   
        3110                 0.302570              0.159409   
...                               ...                   ...   
68631   5632                 0.271691              0.106605   
        3359                 0.079834              0.315965   
        3361                 0.231357              0.166673   
        3314                 0.359384              0.379549   
        5124                 0.300867              0.161063   

                 DistanceTo3thCluster  DistanceTo4thCluster  \
user_id post_id                                               
75106   2061                 0.481784              0.567884   
        3598                 0.256382              0.423852   
        685                  0.411906              0.524848   
        5921                 0.239152              0.463043   
        3110                 0.309820              0.448538   
...                               ...                   ...   
68631   5632                 0.212162              0.397965   
        3359                 0.409165              0.528934   
        3361                 0.341350              0.479208   
        3314                 0.479172              0.586302   
        5124                 0.200507              0.417800   

                 DistanceTo5thCluster  ...  DistanceTo15thCluster  gender  \
user_id post_id                        ...                                  
75106   2061                 0.451066  ...               0.463544       1   
        3598                 0.259322  ...               0.243512       1   
        685                  0.474841  ...               0.439667       1   
        5921                 0.324921  ...               0.275286       1   
        3110                 0.307648  ...               0.302063       1   
...                               ...  ...                    ...     ...   
68631   5632                 0.222199  ...               0.172527       1   
        3359                 0.399419  ...               0.389202       1   
        3361                 0.341685  ...               0.334319       1   
        3314                 0.470502  ...               0.464347       1   
        5124                 0.264031  ...               0.214922       1   

                 age  country      city  exp_group       os  source  hour  \
user_id post_id                                                             
75106   2061      16   Russia  Voronezh          4  Android     ads    22   
        3598      16   Russia  Voronezh          4  Android     ads    22   
        685       16   Russia  Vor

In [56]:
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import pandas as pd

# Проверяем, что X_train и y_train являются DataFrame и Series соответственно
if isinstance(X_train, pd.Series):
    X_train = X_train.to_frame()
if isinstance(y_train, pd.DataFrame):
    y_train = y_train.squeeze()  # Преобразуем в Series, если это DataFrame

object_cols = [
    'topic', 'TextCluster', 'gender', 'country',
    'city', 'exp_group', 'hour', 'month',
    'os', 'source'
]

cols_for_ohe = [x for x in object_cols if X_train[x].nunique() < 5]
cols_for_mte = [x for x in object_cols if X_train[x].nunique() >= 5]

# Сохраним индексы этих колонок 
cols_for_ohe_idx = [list(X_train.columns).index(col) for col in cols_for_ohe]
cols_for_mte_idx = [list(X_train.columns).index(col) for col in cols_for_mte]

# Настройка ColumnTransformer
t = [
    ('OneHotEncoder', OneHotEncoder(), cols_for_ohe_idx),
    ('MeanTargetEncoder', TargetEncoder(), cols_for_mte_idx)
]

col_transform = ColumnTransformer(transformers=t, remainder='passthrough')

# Настройка Pipeline
pipe_dt = Pipeline([
    ("column_transformer", col_transform),
    ("decision_tree", DecisionTreeClassifier())
])

# Обучение модели
pipe_dt.fit(X_train, y_train)


C:\Users\nikul\anaconda3\Lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\nikul\anaconda3\Lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
C:\Users\nikul\anaconda3\Lib\site-packages\sklearn\base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(
C:\Users\nikul\anaconda3\Lib\site-packages\sklearn\base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement th

AttributeError: 'Series' object has no attribute 'iteritems'

In [58]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(iterations=100,
                              learning_rate=1,
                              depth=2)

catboost.fit(X_train, y_train, object_cols)

0:	learn: 0.3566249	total: 1.03s	remaining: 1m 42s
1:	learn: 0.3488835	total: 1.89s	remaining: 1m 32s
2:	learn: 0.3469653	total: 2.65s	remaining: 1m 25s
3:	learn: 0.3461856	total: 3.46s	remaining: 1m 23s
4:	learn: 0.3455331	total: 4.18s	remaining: 1m 19s
5:	learn: 0.3453239	total: 4.97s	remaining: 1m 17s
6:	learn: 0.3452296	total: 5.7s	remaining: 1m 15s
7:	learn: 0.3448510	total: 6.37s	remaining: 1m 13s
8:	learn: 0.3448065	total: 7.08s	remaining: 1m 11s
9:	learn: 0.3447556	total: 7.72s	remaining: 1m 9s
10:	learn: 0.3442814	total: 8.4s	remaining: 1m 7s
11:	learn: 0.3438000	total: 9.07s	remaining: 1m 6s
12:	learn: 0.3433292	total: 9.73s	remaining: 1m 5s
13:	learn: 0.3431965	total: 10.7s	remaining: 1m 5s
14:	learn: 0.3430127	total: 12.5s	remaining: 1m 11s
15:	learn: 0.3427161	total: 14.2s	remaining: 1m 14s
16:	learn: 0.3426688	total: 16.1s	remaining: 1m 18s
17:	learn: 0.3426501	total: 17.8s	remaining: 1m 20s
18:	learn: 0.3425884	total: 19.8s	remaining: 1m 24s
19:	learn: 0.3422768	total: 2

In [62]:
### Замерим качество работы модели 
### ROC-AUC

print(f'Качество на трейне: {roc_auc_score(y_train, catboost.predict_proba(X_train)[:,1])}')
print(f'Качество на тесте: {roc_auc_score(y_test, catboost.predict_proba(X_test)[:,1])}')

Качество на трейне: 0.6675232789298883
Качество на тесте: 0.6483367152138919


In [66]:
### Сохраняем модель 

catboost.save_model(
    "catboost_model_new",
    format = "cbm"
)

In [74]:
# Write processed data to sql
posts_info.to_sql('nm_lesson_22_post', 
                  con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
                      "postgres.lab.karpov.courses:6432/startml", 
                  if_exists='replace', index=False)

23

In [76]:
test = pd.read_sql(
    """SELECT * FROM public.nm_lesson_22_post""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
)
test.head()

,post_id,text,topic,TotalTfidf,MaxTfidf,MeanTfidf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,...,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8.748129,0.495805,0.000190,4,0.498763,0.438866,0.478928,...,0.501817,0.549847,0.407473,0.441331,0.465443,0.444900,0.529236,0.505747,0.522130,0.437839
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11.878472,0.308003,0.000258,7,0.371631,0.291801,0.341899,...,0.375288,0.450040,0.173449,0.292427,0.329292,0.282600,0.414625,0.426678,0.413365,0.259837
2,3,Asian quake hits European shares\n\nShares in ...,business,12.675530,0.261799,0.000276,4,0.397930,0.313296,0.362400,...,0.389489,0.484424,0.279320,0.315376,0.356818,0.326329,0.431981,0.427459,0.435588,0.309584
3,4,India power shares jump on debut\n\nShares in ...,business,6.622786,0.537713,0.000144,4,0.323621,0.256363,0.321060,...,0.352320,0.447335,0.200471,0.263464,0.299496,0.287838,0.385666,0.403035,0.399316,0.257896
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,6.352096,0.420251,0.000138,7,0.294497,0.160849,0.260119,...,0.296027,0.405016,0.139676,0.179389,0.234276,0.219992,0.343986,0.352692,0.391472,0.182880
